<a href="https://colab.research.google.com/github/pssr1183/Text-Analytics-Model/blob/main/Text_Analytics_sequential_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 41.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv")

# Preprocessing
data['review'] = data['review'].str.replace('<.*?>', '', regex=True)  # Remove HTML tags
data['review'] = data['review'].str.replace('[^\w\s]', '')  # Remove punctuation
data['review'] = data['review'].str.lower()  # Convert to lowercase

# Split the dataset into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(data['review'], data['sentiment'], test_size=0.2, random_state=42)


<ipython-input-3-6ed6380fab51>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  data['review'] = data['review'].str.replace('[^\w\s]', '')  # Remove punctuation


In [ ]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
import numpy as np

# Tokenization
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

# Padding sequences
max_sequence_length = 200
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Load pre-trained word embeddings (GloVe)
embedding_dim = 100
embeddings_index = {}
with open('/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalAveragePooling1D
import numpy as np

# ... (previous code for loading data and preprocessing)

# Convert labels to float32
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

# Build the model
model = Sequential([
    Embedding(max_words, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False),
    LSTM(128, return_sequences=True),
    GlobalAveragePooling1D(),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 128
epochs = 5
history = model.fit(train_padded, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(test_padded, test_labels))


Epoch 1/5
313/313 [==============================] - 239s 756ms/step - loss: 0.5163 - accuracy: 0.7436 - val_loss: 0.4254 - val_accuracy: 0.8048
Epoch 2/5
313/313 [==============================] - 214s 684ms/step - loss: 0.4026 - accuracy: 0.8167 - val_loss: 0.3798 - val_accuracy: 0.8279
Epoch 3/5
313/313 [==============================] - 214s 684ms/step - loss: 0.3599 - accuracy: 0.8427 - val_loss: 0.3535 - val_accuracy: 0.8450
Epoch 4/5
313/313 [==============================] - 236s 754ms/step - loss: 0.3382 - accuracy: 0.8518 - val_loss: 0.3291 - val_accuracy: 0.8554
Epoch 5/5
313/313 [==============================] - 220s 703ms/step - loss: 0.3212 - accuracy: 0.8623 - val_loss: 0.4059 - val_accuracy: 0.8201


In [ ]:
# Convert labels to numerical values
train_labels = train_labels.apply(lambda x: 0 if x == 'negative' else 1)
test_labels = test_labels.apply(lambda x: 0 if x == 'negative' else 1)

# Convert labels to float32
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)


In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_padded, test_labels)
print("Test accuracy:", test_acc)

313/313 [==============================] - 36s 115ms/step - loss: 0.4059 - accuracy: 0.8201
Test accuracy: 0.8201000094413757
1/1 [==============================] - 1s 810ms/step
Review: This movie was fantastic! Highly recommended.
Sentiment: Positive

Review: Terrible movie. Waste of time.
Sentiment: Negative



In [ ]:

# Threshold to classify reviews as positive or negative
threshold = 0.4

# List of movie reviews for a single movie
movie_reviews = [
    "This movie was absolutely amazing! I was on the edge of my seat the whole time.",
    "The acting in this film was terrible, and the plot made no sense.",
    "I really enjoyed the visuals and special effects, but the story was lacking.",
    "I can't believe I wasted my time watching this. It was so boring.",
    "A masterpiece of storytelling and cinematography. A must-see!",
]

# Predict sentiment for the movie reviews with negation handling
movie_sequences = tokenizer.texts_to_sequences(movie_reviews)
movie_padded = pad_sequences(movie_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
predictions = model.predict(movie_padded)

# Calculate average sentiment score for the reviews
average_sentiment = sum(predictions) / len(predictions)

# Assign ratings based on average sentiment
if average_sentiment >= threshold:
    rating = 5 if average_sentiment >= 0.7 else 4  # High rating for strong positive sentiment
else:
    rating = 2 if average_sentiment >= 0.3 else 1  # Low rating for strong negative sentiment

# Print the results
print("Movie Reviews:")
for review, sentiment in zip(movie_reviews, predictions):
    sentiment_label = 'Positive' if sentiment >= threshold else 'Negative'
    print(f"Review: {review}\nSentiment: {sentiment_label}\n")

print("Average Sentiment:", average_sentiment)
print("Rating:", rating, "/5")


1/1 [==============================] - 0s 38ms/step
Movie Reviews:
Review: This movie was absolutely amazing! I was on the edge of my seat the whole time.
Sentiment: Positive

Review: The acting in this film was terrible, and the plot made no sense.
Sentiment: Negative

Review: I really enjoyed the visuals and special effects, but the story was lacking.
Sentiment: Positive

Review: I can't believe I wasted my time watching this. It was so boring.
Sentiment: Negative

Review: A masterpiece of storytelling and cinematography. A must-see!
Sentiment: Positive

Average Sentiment: [0.58707607]
Rating: 4 /5


In [ ]:
model.save("model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
